In [1]:
#NHDPlusAttributes.dbf append all regions and push from SB to GC2

import pysb
import requests, json, os
import urllib.request as ur
import pandas as pd
import time
import geopandas as gpd

regList = ['Reg1','Reg2','Reg3','Reg4','Reg5','Reg6','Reg7','Reg8']
#Need to write in a check here to make sure the right number of files are returned (21)
q="""https://www.sciencebase.gov/catalog/items?filter=tags={%22scheme%22:%22BIS%22,%22name%22:%22NHDPlusV1%22}&fields=files,id,tags&max=40&format=json"""
#Returns 
nhdItems = requests.get(q).json()
#print (nhdItems)

dfAll = None

for item in nhdItems['items']:
    #Identify which NHD Region via tags 
    for tag in item['tags']:
        if 'Reg' in tag['name']:
            region = tag['name']
            if region in regList:
                time.sleep(2)
                #Look at files and find files for NHDFLOWLINE
                for file in item['files']:
                    fileName = str.lower(file['name'])
                    if 'nhdflowline.' in fileName:
                        print ('Retrieving region ' + region + ', file:' + fileName)
                        fileUrl = file['url']
                        try:
                            ur.urlretrieve(fileUrl, fileName)    
                        except:
                            print (region + " broke")
                if dfAll is None:
                    dfAll = gpd.read_file('nhdflowline.shp', crs=4269)
                else:
                    dfReg = gpd.read_file('nhdflowline.shp', crs=4269)
                    dfAll = dfAll.append(dfReg)
                os.remove('nhdflowline.shp')
                os.remove('nhdflowline.prj')
                os.remove('nhdflowline.dbf')
                os.remove('nhdflowline.shx')



Retrieving region Reg4, file:nhdflowline.dbf
Retrieving region Reg4, file:nhdflowline.prj
Retrieving region Reg4, file:nhdflowline.shp
Retrieving region Reg4, file:nhdflowline.shx
Retrieving region Reg6, file:nhdflowline.dbf
Retrieving region Reg6, file:nhdflowline.prj
Retrieving region Reg6, file:nhdflowline.shp
Retrieving region Reg6, file:nhdflowline.shx
Retrieving region Reg1, file:nhdflowline.dbf
Retrieving region Reg1, file:nhdflowline.prj
Retrieving region Reg1, file:nhdflowline.shp
Retrieving region Reg1, file:nhdflowline.shx
Retrieving region Reg7, file:nhdflowline.dbf
Retrieving region Reg7, file:nhdflowline.prj
Retrieving region Reg7, file:nhdflowline.shp
Retrieving region Reg7, file:nhdflowline.shx
Retrieving region Reg8, file:nhdflowline.dbf
Retrieving region Reg8, file:nhdflowline.prj
Retrieving region Reg8, file:nhdflowline.shp
Retrieving region Reg8, file:nhdflowline.shx
Retrieving region Reg5, file:nhdflowline.dbf
Retrieving region Reg5, file:nhdflowline.prj
Retrieving

In [2]:
dfAll.shape

(1351295, 14)

In [5]:
dfAll.drop('RESOLUTION', axis=1, inplace=True)
dfAll.drop('SHAPE_LENG', axis=1, inplace=True)
dfAll.drop('WBAREACOMI', axis=1, inplace=True)

In [6]:
dfAll.head()

,COMID,ENABLED,FCODE,FDATE,FLOWDIR,FTYPE,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,geometry
0,271811,T,46006,2004-08-01,With Digitized,StreamRiver,,,0.084,04030112000113,LINESTRING Z (-86.76509310909636 46.3023853281...
1,271813,T,46006,2004-08-01,With Digitized,StreamRiver,,,0.520,04030112000111,LINESTRING Z (-86.74513070912734 46.2666033281...
2,271815,T,46006,2004-08-01,With Digitized,StreamRiver,01621807,Sturgeon River,4.492,04030112000057,LINESTRING Z (-86.76653030909412 46.2927175948...
3,271817,T,46006,2004-08-01,With Digitized,StreamRiver,,,1.764,04030112000115,LINESTRING Z (-86.78428457573322 46.2642741948...
4,271819,T,46006,2004-08-01,With Digitized,StreamRiver,,,0.552,04030112000109,LINESTRING Z (-86.75561790911107 46.2610477948...


In [7]:
#Function to process pandas dataframe row into a dictionary

def pandasRowToDict(df, r):   #df = pandas dataframe, r = index number for row to process
    import datetime
    import copy
    intList = ['COMID','FCODE']
    c = 0 #Start at column index = 0
    cMax = len(df.columns) #Number of columns in df 
    record = {} 
    while c < cMax:
        if df.columns[c] in intList:
            record[(df.columns[c])] = int(df.iloc[r,c])
            c += 1
        else:
            record[(df.columns[c])] = df.iloc[r,c]
            c += 1
        
    return record





gc2Data = []
r = 0
rMax = len(dfAll)
#rMax = 8

startTime = time.clock()

#For each feature within the dataset build a dictionary and append to a list
while r < rMax:
    record = pandasRowToDict (alaska_dissolve, r)    
    record['_id'] = str(uuid.uuid4())
    record['regDate'] = datetime.datetime.utcnow().isoformat() 
    record['regCode'] = 'https://github.com/dwief-usgs/sfrTesting/blob/master/Level3_EcoRegions_Into_SFR.ipynb'
    record['fType'] = 'NHDPlusV1 Flowline'
    record['spatCert'] = 'https://www.sciencebase.gov/vocab/term/5822257ee4b0b3d9add24304'
    r += 1
    gc2Data.append(record)
    
gc2Df = gpd.GeoDataFrame(gc2Data, geometry = 'geometry', crs='4269')

for row in gc2Data:
    try:
        del row['geometry']
    except KeyError:
        pass
    
print (str(time.clock()-startTime/60))  #Print processing time in minutes

NameError: name 'alaska_dissolve' is not defined

In [ ]:
dfAll.to_file('nhdplusv1_.shp', driver='ESRI Shapefile', crs_wkt='4269')